# 02 - Summarising segments
Often when we are doing analytics, we need to segment our data and summarise those segments so they can be compared and decisions can be made. For example, to help us decide which segment to target for a marketing campaign.

In these exercises we are going to explore some segments of the listings data and create some stats for each segement. Let's first import `pandas` and load the listings data.

In [1]:
import pandas as pd
df_listings = pd.read_csv('../data/wk3_listings_sample.csv')

## Neighborhood segments
As a first example, let's assume we want to see how many listings in the dataset are based in each neighbourhood in the dataset.

Using tools from before, we can do this quite simply using the `value_counts()` function. Run the code below.

In [2]:
df_listings['neighbourhood_cleansed'].value_counts().head()

Westminster               3359
Tower Hamlets             2603
Hackney                   2254
Kensington and Chelsea    2175
Camden                    2065
Name: neighbourhood_cleansed, dtype: int64

**Question:** Which neighbourhood in the dataset has the most listings in the dataset?

### Exercise-01: Number of listings

If we wanted to do this a bit more cleanly, and create a new `DataFrame` showing each neighborhood and the number of listings in each neighbourhood, then we would run code like shown below.

In [3]:
df_neighborhood_counts = df_listings['neighbourhood_cleansed'].value_counts().reset_index()
df_neighborhood_counts.columns = ['neighbourhood_cleansed','number_of_listings']
df_neighborhood_counts.head()

,neighbourhood_cleansed,number_of_listings
0,Westminster,3359
1,Tower Hamlets,2603
2,Hackney,2254
3,Kensington and Chelsea,2175
4,Camden,2065


### Exercise-02: Least listings
If we have a `DataFrame` containing the number of listings for each neighbourhood, then we can easily identify the neighbourhood which the least listings. Sort the rows in `df_neighborhood_counts` by the values in `number_of_listings` to identify the neighbourhood in the dataset with the least number of listings in the dataset.

In [4]:
# (SOLUTION)
df_neighborhood_counts.sort_values(by='number_of_listings').head()

,neighbourhood_cleansed,number_of_listings
32,Havering,103
31,Sutton,114
30,Bexley,115
29,Barking and Dagenham,157
28,City of London,158


**Question:** Which neighbourhood in the dataset has the least number of listings in the dataset?

## Neighbourhood-level aggregates
We are now going to explore a new (more extensible) way to compute statistics for for each segment (e.g. neighbourhood) in a dataset. First, let's count the number of listings in each neighbourhood using a different approach to the one used in Exercise-01 (i.e. without using the `value_counts()` method).

### Exercise-03: Count listings
To create a `DataFrame` containing the number of listings for each neighbourhood, without using the `value_counts()` function, we need to:
1. Use the `groupby` method to group the rows of `df_listings` by `neighbourhood_cleansed`.
2. Then use the `agg` method and `count` function to count the number of rows in each group.
3. Then clean-up the columns names of resulting `DataFrame`.
4. Finally, sort the rows of the resulting `DataFrame` and show its `head`.

In [5]:
df_listings_grouped = df_listings.groupby('neighbourhood_cleansed')
df_listings_aggregated = df_listings_grouped['id'].agg(['count']).reset_index()
df_listings_aggregated = df_listings_aggregated.rename(columns={'count': 'number_of_listings'})
df_listings_aggregated.sort_values(by='number_of_listings', ascending=False).head()

,neighbourhood_cleansed,number_of_listings
32,Westminster,3359
29,Tower Hamlets,2603
11,Hackney,2254
19,Kensington and Chelsea,2175
5,Camden,2065


**Question:** Is `df_listings_aggregated` the same as `df_neighborhood_counts`? 

### Exercise-04: Mean price
As stated, the method used in Exercise-03 is extendable to other, similar, analytics problems. For example, let's try to use the `groupby` and `agg` methods to estimate the mean price of listings within each neighbourhood.

First, let's format the `price` column in `df_listings` again! To do this, run the code below.

In [6]:
def format_price(price):
    return(float(price.replace('$','').replace(',','')))

# (SOLUTION)
df_listings['price_$'] = df_listings['price'].apply(format_price) 
df_listings[['price','price_$']].head()

,price,price_$
0,$128.00,128.0
1,$70.00,70.0
2,$39.00,39.0
3,$85.00,85.0
4,$40.00,40.0


**Question:** What might you do to avoid having to format the price column repeatability?

Great! If we have formated the `price` column correctly, we will be able to calculate the mean price for each neighbourhood. Using Pandas, we will do this in three steps:

1. Use the `groupby` method to group the rows of `df_listings` by `neighbourhood_cleansed`.
2. Then use the `agg` method and `mean` function to calculate the mean `price_$` for each group.
3. Then clean-up the columns names of resulting `DataFrame`.
4. Finally, sort the rows of the resulting `DataFrame` and show its `head`.

In [7]:
df_listings_grouped = df_listings.groupby('neighbourhood_cleansed')
df_listings_aggregated = df_listings_grouped['price_$'].agg(['mean']).reset_index()
df_listings_aggregated = df_listings_aggregated.rename(columns={'mean': 'mean_price_$'})
df_listings_aggregated.sort_values(by='mean_price_$', ascending=False).head()

,neighbourhood_cleansed,mean_price_$
6,City of London,201.936709
32,Westminster,185.473355
19,Kensington and Chelsea,183.262529
5,Camden,138.333172
18,Islington,132.304448


**Question:** Which neighbourhood has the highest `mean_price_$`?

### Exercise-05: Mean price per person
Use what you have learned here and in notebook `Exercise-02` of notebook `01 - Calculating metrics` to estimate the mean `price_$/person` for each neighbourhood.

In [8]:
# (SOLUTION)

### Exercise-06: Sum of forecast income
Use what you have learned here and in notebook `Exercise-03` of notebook `01 - Calculating metrics` to forecast the total income over the next 30 days (e.g. `estimated_income_30`) for each neighbourhood.

*Hint*: You will need to use `sum` as the function used to aggregate, rather than `mean`. 

In [9]:
# (SOLUTION)

## Bonus exercises
Some bonus exercises!

### Exercise-07: Even more extensible code
We can write the code in `Exercise-04` in an *even more* extensible way.

In [10]:
group_col = 'neighbourhood_cleansed'
agg_col = 'price_$'
agg_func = 'mean'

df_listings_grouped = df_listings.groupby(group_col)
df_aggregated = df_listings_grouped[agg_col].agg([agg_func]).reset_index()
df_aggregated = df_aggregated.rename(columns={agg_func: f'{agg_func}_{agg_col}'})
df_aggregated.head()

,neighbourhood_cleansed,mean_price_$
0,Barking and Dagenham,57.662420
1,Barnet,99.901235
2,Bexley,56.165217
3,Brent,101.111588
4,Bromley,83.211864


Here we have remove specifications out from the main code. Play around with other types of groupings and aggregations.

### Exercise-08: Data cleaning
Did you notice that we've been using the values in `neighbourhood_cleansed` to compute the neighbourhood of each listing? What do you think `cleansed` means? Are you able to find the answer in the [data dictionary](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=982310896)? Why do you think we didn't use that values in in `neighbourhood`?